In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import train_test_split
import seaborn as sns # for plotting.
import os
import math
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [2]:
def preprocess(df, name):
    df_gyro = pd.DataFrame(columns=['time','GY(x)', 'GY(y)', 'GY(z)'])
    df_acc = pd.DataFrame(columns=['time','LA(x)', 'LA(y)', 'LA(z)'])
    
    df_gyro[['time','GY(x)', 'GY(y)', 'GY(z)']] = df[['TimeStamp','GY(x)', 'GY(y)', 'GY(z)']].copy()
#     df_gyro['name'] = df_gyro['name'].fillna(name)
    df_gyro = df_gyro.dropna()
    df_acc[['time','LA(x)', 'LA(y)', 'LA(z)']] = df[['TimeStamp','LA(x)', 'LA(y)', 'LA(z)']].copy()
#     df_acc['name'] = df_acc['name'].fillna(name)
    df_acc = df_acc.dropna()
    
    df_gyro['time'] = pd.to_datetime(df_gyro['time'], unit='ms')
    df_indexed_gyro = df_gyro.set_index('time')
    resampled_df_gyro = df_indexed_gyro.resample('5ms').mean().interpolate()
    resampled_df_gyro['name'] = name
    resampled_df_gyro.reset_index(inplace=True)
    
    df_acc['time'] = pd.to_datetime(df_acc['time'], unit='ms')
    df_indexed_acc = df_acc.set_index('time')
    resampled_df_acc = df_indexed_acc.resample('5ms').mean().interpolate()
#     resampled_df_acc['name'] = name
    resampled_df_acc.reset_index(inplace=True)
    
    # Inner join: Only the common values in both DataFrames are included in the merged DataFrame (inner_merged_df).
    
    inner_merged_df = pd.merge(resampled_df_acc, resampled_df_gyro, on='time', how='inner')
    
    return inner_merged_df

In [3]:
df_final = pd.DataFrame(columns=['name','time','LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)'])
directory = os.path.join(os.getcwd(), 'straightWalk')
x=1
firstTime = True
for filename in os.listdir(directory):
    if (filename.endswith(".csv")):
        filepath = os.path.join(directory, filename)
        df_original = pd.read_csv(filepath)
        
#         num_rows_before = df_original.shape[0]
    
#         twenty = num_rows_before * 0.20
#         twenty = math.ceil(twenty)
        
#         #test = test.iloc[fivePercent:]
#         df_original_remove20 = df_original.iloc[:-twenty]
        
         
       
        df_processed = preprocess(df_original, filename[:-4])

        df_final = pd.concat([df_final, df_processed])
        
df_final

,name,time,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z)
0,alibadawy,1970-01-01 00:00:00.000,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138
1,alibadawy,1970-01-01 00:00:00.005,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157
2,alibadawy,1970-01-01 00:00:00.010,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000
3,alibadawy,1970-01-01 00:00:00.015,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288
4,alibadawy,1970-01-01 00:00:00.020,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030
...,...,...,...,...,...,...,...,...
5275,zoz,1970-01-01 00:00:26.375,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494
5276,zoz,1970-01-01 00:00:26.380,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391
5277,zoz,1970-01-01 00:00:26.385,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786
5278,zoz,1970-01-01 00:00:26.390,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156


In [4]:
df_Xtime = df_final.drop(['time'], axis = 1).copy()
df_Xtime.shape

(155302, 7)

In [5]:
label = LabelEncoder()
df_Xtime['label'] = label.fit_transform(df_Xtime['name'])
df_Xtime

,name,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,alibadawy,-0.268265,-0.005538,-0.071094,-0.016798,0.103233,0.033138,0
1,alibadawy,-0.163861,-0.004340,0.122527,-0.017613,0.110563,0.034157,0
2,alibadawy,-0.018824,-0.011455,0.059537,-0.018936,0.123697,0.031000,0
3,alibadawy,0.043948,-0.010231,0.135187,-0.019445,0.136423,0.030288,0
4,alibadawy,0.106720,-0.009007,0.210836,-0.013744,0.146603,0.027030,0
...,...,...,...,...,...,...,...,...
5275,zoz,0.032646,0.028275,0.519662,0.135150,-0.022907,0.223494,30
5276,zoz,-0.478584,-0.296353,0.069982,0.152254,-0.098041,0.268391,30
5277,zoz,-0.897290,-0.563466,-0.287334,0.155308,-0.165234,0.318786,30
5278,zoz,-1.035197,-0.655348,-0.474507,0.145534,-0.212269,0.362156,30


In [6]:
X = df_Xtime[['LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)']].copy()
y = df_Xtime['label'].copy()

In [7]:
scaler = StandardScaler()
X_transformed = X.copy()
X_transformed = scaler.fit_transform(X_transformed)

scaled_X = pd.DataFrame(data = X_transformed, columns = ['LA(x)', 'LA(y)', 'LA(z)','GY(x)', 'GY(y)', 'GY(z)'])
scaled_X['label'] = y.values

scaled_X

,LA(x),LA(y),LA(z),GY(x),GY(y),GY(z),label
0,-0.157996,0.116219,-0.109081,-0.070382,0.176505,0.142357,0
1,-0.120536,0.116679,-0.048151,-0.071206,0.181998,0.143630,0
2,-0.068498,0.113950,-0.067973,-0.072545,0.191841,0.139684,0
3,-0.045976,0.114420,-0.044167,-0.073060,0.201378,0.138793,0
4,-0.023454,0.114889,-0.020362,-0.067292,0.209007,0.134721,0
...,...,...,...,...,...,...,...
155297,-0.050031,0.129187,0.076822,0.083323,0.081973,0.380324,30
155298,-0.233456,0.004690,-0.064686,0.100625,0.025666,0.436450,30
155299,-0.383684,-0.097750,-0.177129,0.103714,-0.024689,0.499450,30
155300,-0.433164,-0.132987,-0.236030,0.093828,-0.059938,0.553668,30


In [8]:
import scipy.stats as stats

Fs = 200
frame_size = Fs*6
hop_size = int(Fs*1.5)

In [9]:
df_framed = pd.DataFrame(columns=['name','mean_LA(x)', 'mean_LA(y)', 'mean_LA(z)', 'mean_GY(x)', 'mean_GY(y)', 'mean_GY(z)','sd_LA(x)', 'sd_LA(y)', 'sd_LA(z)', 'sd_GY(x)', 'sd_GY(y)', 'sd_GY(z)'])
def get_frames(df, frame_size, hop_size):
    
    last_value = df['label'].iloc[-1]
    for j in range (0,last_value+1): 
        filtered_df = df[df['label'] == j]
        filtered_df = filtered_df.drop('label', axis = 1)
        for i in range(0, len(filtered_df) - frame_size, hop_size):
            frame = filtered_df[i: i + frame_size]

            new_row = [j
                       ,frame['LA(x)'].mean(), frame['LA(y)'].mean(), frame['LA(z)'].mean(),
                       frame['GY(x)'].mean(), frame['GY(y)'].mean(), frame['GY(z)'].mean(),
                       frame['LA(x)'].std(), frame['LA(y)'].std(), frame['LA(z)'].std(),
                       frame['GY(x)'].std(), frame['GY(y)'].std(), frame['GY(z)'].std()]



            df_framed.loc[len(df_framed)] = new_row


    return df_framed

In [10]:
df_framed = get_frames(scaled_X,frame_size,hop_size)
df_framed

,name,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
0,0.0,-0.173025,0.002897,-0.023928,0.006710,0.069904,0.025017,0.971869,0.729274,0.982268,0.937521,0.783991,0.816932
1,0.0,-0.116492,-0.063763,0.119805,-0.009734,-0.256438,-0.083630,1.114540,0.766249,1.125036,1.056636,1.049760,0.932551
2,0.0,-0.138057,-0.043222,0.188650,0.028022,-0.299930,-0.093680,1.167728,0.845961,1.177114,1.074790,1.054738,0.997421
3,0.0,-0.085503,-0.077410,0.205353,0.046562,-0.283551,-0.076235,1.128992,0.805893,1.185070,1.093444,1.088436,0.982225
4,0.0,0.019135,-0.093058,0.239105,0.014979,-0.504918,-0.095435,1.120264,0.776490,1.195407,1.076988,1.229421,0.958123
...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,30.0,-0.066722,0.118244,-0.100634,-0.228716,-0.271690,-0.028413,0.781184,0.833872,0.697507,0.840068,0.833594,0.693555
406,30.0,-0.088812,0.124968,-0.139645,-0.213767,-0.282085,-0.057373,0.720295,0.784162,0.688677,0.841758,0.849291,0.667307
407,30.0,-0.047925,0.104805,-0.086457,-0.227873,-0.298455,-0.072681,0.585293,0.655446,0.577344,0.763494,0.709827,0.538569
408,30.0,-0.064867,0.145250,-0.113207,-0.191153,-0.245996,0.068315,0.547353,0.578936,0.529081,0.676879,0.702108,0.603658


In [11]:
occurrences = df_framed['name'].value_counts()
print(occurrences.mean())

13.225806451612904


In [12]:
X = df_framed.drop('name', axis=1)  # Features (all columns except the target)
y = df_framed['name']  # Labels

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0, stratify = y)

In [14]:
X_test

,mean_LA(x),mean_LA(y),mean_LA(z),mean_GY(x),mean_GY(y),mean_GY(z),sd_LA(x),sd_LA(y),sd_LA(z),sd_GY(x),sd_GY(y),sd_GY(z)
295,-0.076415,-0.120586,-0.358554,-0.066904,-0.255339,0.342717,1.079882,1.489718,1.505623,1.477025,1.086593,0.737497
385,-0.068624,0.306719,0.220945,0.210196,0.489859,0.039497,1.024063,1.530835,1.645057,1.850922,1.711570,0.855187
35,-0.044827,0.029364,0.152084,-0.018693,-0.319101,0.007792,0.966090,0.837875,0.813166,0.705358,0.996854,0.722806
261,-0.102739,-0.002324,0.072701,0.013336,-0.144307,0.064639,1.558765,0.730655,0.597640,0.589498,1.077438,1.594164
214,-0.045953,-0.142285,0.030059,-0.218504,0.461881,0.033090,2.045655,1.382139,1.288567,1.089395,1.302816,1.292218
...,...,...,...,...,...,...,...,...,...,...,...,...
8,-0.016381,-0.071575,0.160657,0.004451,-0.315182,-0.109746,1.133408,0.864444,1.141012,1.072456,1.061381,0.948036
94,0.026111,0.028888,-0.043707,-0.023482,0.506580,0.091891,0.739144,0.927686,0.754006,0.878004,0.964933,0.394949
376,0.034558,0.002143,-0.137671,-0.201468,0.491277,0.071650,0.891325,0.920668,1.056971,0.972749,0.806920,0.552940
166,-0.044715,0.088277,0.064801,0.110978,-0.149504,0.013315,0.534711,0.628285,0.436680,0.551548,0.651395,0.369433


In [15]:
occurrences = y_test.value_counts()
print(occurrences.mean())

3.967741935483871


In [16]:
X_train.shape, X_test.shape

((287, 12), (123, 12))

In [17]:
y_train.shape, y_test.shape

((287,), (123,))

In [18]:
#Create a svm Classifier
model = svm.SVC(C = 15, # reg paramater
                kernel='linear', #kernel{‘linear’, ‘poly’, ‘rbf’, ‘sigmoid’, ‘precomputed’}, default=’rbf’
               ) # Linear Kernel

#Train the model using the training sets
model.fit(X_train, y_train)


from sklearn import metrics
y_pred = model.predict(X_test)
print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.9186991869918699
